In [1]:
!pip install pycm livelossplot
%pylab inline

You should consider upgrading via the '/opt/firedrake/bin/python -m pip install --upgrade pip' command.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *
from PIL import Image
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

In [3]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = torch.device("cuda:0")
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


In [4]:
import glob
from PIL import Image
import numpy as np
import pandas as pd
import json 


with open("mapping.json",'r', encoding='UTF-8') as f:
     load_dict = json.load(f)


train_file = glob.glob('train/*/images/*.JPEG')
print(len(train_file))
train_data = []
labels = []
count = 0


for f in train_file:
    img = np.array(Image.open(f))
    label_name = f.split('/')[1]
    label = load_dict.get(label_name)
    if img.shape != (64,64,3):
        img = np.stack((img,)*3, axis=-1)
    
    train_data.append(img)
    labels.append(label)
    count += 1

print(len(train_data))

100000
100000


In [5]:
train_data = np.array(train_data)
labels = np.array(labels)
print(train_data.shape)
print(labels.shape)

(100000, 64, 64, 3)
(100000,)


In [6]:
mean = train_data.mean(axis=(0,1,2))/255
std = train_data.std(axis=(0,1,2))/255

print(mean)
print(std)

[0.48024579 0.44807218 0.39754775]
[0.27698641 0.26906449 0.28208191]


In [7]:
shuffler = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42).split(train_data, labels)

indices = [(train_idx, validation_idx) for train_idx, validation_idx in shuffler][0]
#valid_idx = list(set(train_idx).difference(set(train_dataset)))

print(len(indices[0]))
print(len(indices[1]))

90000
10000


In [8]:
X_train, y_train = torch.tensor(train_data[indices[0]]).float(), torch.tensor(labels)[indices[0]]
X_val, y_val = torch.tensor(train_data[indices[1]]).float(), torch.tensor(labels)[indices[1]]


X_train = X_train.permute(0,3,1,2)
X_val = X_val.permute(0,3,1,2)


print(X_train.size())
print(X_val.size())
print(y_train.size())

torch.Size([90000, 3, 64, 64])
torch.Size([10000, 3, 64, 64])
torch.Size([90000])


In [9]:
from torch.utils.data import Dataset 

class CustomImageTensorDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        """
        Args:
            data (Tensor): A tensor containing the data e.g. images
            targets (Tensor): A tensor containing all the labels
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data
        self.targets = targets
        self.transform = transform


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        sample, label = self.data[idx], self.targets[idx]
        #sample = sample.view(3, 64, 63).float()/255.
        if self.transform:
            sample = self.transform(sample)


        return sample, label

In [10]:
train_transform = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256,256)),
    transforms.RandomCrop(224, padding=4),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

valid_transform = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

train_set = CustomImageTensorDataset(X_train, y_train.long(), transform=train_transform)
valid_set = CustomImageTensorDataset(X_val, y_val.long(), transform=valid_transform)

In [11]:
# Set hyperparameters

seed = 42
lr = 1e-2
momentum = 0.9
batch_size = 32
test_batch_size = 1000
n_epochs = 90

In [12]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=test_batch_size, shuffle=False, num_workers=0)

In [13]:
def train(model, optimizer, criterion, data_loader):
    model.train() ## the model is in the training mode so the parameters(weights)to be optimised will be updatad at each step
    train_loss, train_accuracy = 0, 0
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        a2 = model(X.view(-1, 3, 224, 224))
        loss = criterion(a2, y)
        loss.backward()
        train_loss += loss*X.size(0)
        y_pred = F.log_softmax(a2, dim=1).max(1)[1]
        train_accuracy += accuracy_score(y.cpu().numpy(), y_pred.detach().cpu().numpy())*X.size(0) ## .cpu cpoies the tensor to the cpu to evaluate the accuracy score since it is a scikitlearn metric which does not run on GPUs
        optimizer.step()  
        
    return train_loss/len(data_loader.dataset), train_accuracy/len(data_loader.dataset)  ## loss and accuracy calculated give the total for all batches and must be divided by the number of batch size to give the average values.

def validate(model, criterion, data_loader):
    model.eval() ## model is set to evaluation mode to freeze the parameters and ensure weights are not updated at each step so that the trained model is used to evaluate the validation loss
    validation_loss, validation_accuracy = 0., 0.
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model(X.view(-1, 3, 224, 224))
            loss = criterion(a2, y)
            validation_loss += loss*X.size(0)
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            validation_accuracy += accuracy_score(y.cpu().numpy(), y_pred.cpu().numpy())*X.size(0)
            
    return validation_loss/len(data_loader.dataset), validation_accuracy/len(data_loader.dataset)

In [14]:
def evaluate(model, data_loader):
    model.eval()
    ys, y_preds = [], []
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model(X.view(-1, 3, 64, 64))
            #a2 = model(X.view(-1, 28*28)) #What does this have to look like for our conv-net? Make the changes!
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            ys.append(y.cpu().numpy())
            y_preds.append(y_pred.cpu().numpy())
            
    return np.concatenate(y_preds, 0),  np.concatenate(ys, 0)

In [15]:
from __future__ import print_function, division
 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

plt.ion()

In [16]:
def train_model(model, criterion, optimizer, lr=1e-2, num_epochs=30):
 
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_set, batch_size=test_batch_size, shuffle=False, num_workers=0)
  
    liveloss = PlotLosses()
    for epoch in range(num_epochs):
        if epoch == 60 or epoch == 75:
            lr *= 0.1
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
        
        logs = {}
        train_loss, train_accuracy = train(model, optimizer, criterion, train_loader)

        logs['' + 'log loss'] = train_loss.item()
        logs['' + 'accuracy'] = train_accuracy.item()

        validation_loss, validation_accuracy = validate(model, criterion, valid_loader)
        logs['val_' + 'log loss'] = validation_loss.item()
        logs['val_' + 'accuracy'] = validation_accuracy.item()

        liveloss.update(logs)
        liveloss.draw()

    return model

In [21]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        # 224x224
        self.c1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.c2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.s3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 112x112
        self.c4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.c5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.s6 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 56x56
        self.c7 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.c8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.s9 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 28x28
        self.c10 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.c11 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.s12 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 14x14
        self.c13 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.c14 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.s15 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 7x7
        self.s16 = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        # 4x4
        self.f17 = nn.Linear(8192, 2048)
        self.output = nn.Linear(2048, 200)
        self.act = nn.PReLU()
        self.bn = nn.BatchNorm1d(8192)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.act(self.c1(x))
        x = self.act(self.c2(x))
        x = self.act(self.s3(x))
        x = self.act(self.c4(x))
        x = self.act(self.c5(x))
        x = self.act(self.s6(x))
        x = self.act(self.c7(x))
        x = self.act(self.c8(x))
        x = self.act(self.s9(x))
        x = self.act(self.c10(x))
        x = self.act(self.c11(x))
        x = self.act(self.s12(x))
        x = self.act(self.c13(x))
        x = self.act(self.c14(x))
        x = self.act(self.s15(x))
        x = self.act(self.s16(x))
        x = x.view(-1, 8192)  ##returns a "flattened" view of the 2d tensor as inputs for the fully connected layer
        x = self.bn(x)
        x = self.act(self.f17(x))
        x = self.dropout(x)
        return self.output(x)

In [24]:
model = VGG19().to(device)
criterion = nn.CrossEntropyLoss()

# Xavier Initialization
def weight_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
        nn.init.zeros_(m.bias)

model.apply(weight_init)

optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)
model = nn.DataParallel(model)

RuntimeError: CUDA error: out of memory

In [ ]:
model = train_model(model, criterion, optimizer, num_epochs=n_epochs)

In [ ]:
data_path='test/'

test_transform = torchvision.transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])

test_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=test_transform
    )

import glob
from PIL import Image
import numpy as np
import pandas as pd
import json 

 

test_file = glob.glob('test/images/*.JPEG')
print(len(test_file))
test_data = []
test_names = []

 

for f in test_file:
    img = np.array(Image.open(f))
    test_name = f.split('/')[2][5:-5]
    #label = load_dict.get(label_name)
    if img.shape != (64,64,3):
        img = np.stack((img,)*3, axis=-1)
    
    test_data.append(img)
    test_names.append(int(test_name))
 
#y_test is only used for generating dataset. it's not the real labels
y_test = np.array(test_names)
y_test = torch.tensor(y_test)
 
X_test = np.array(test_data)
X_test = torch.tensor(X_test).float()
X_test = X_test.permute(0,3,1,2)
 
test_set = CustomImageTensorDataset(X_test, y_test.long(), transform=valid_transform)

# test_data = torch.Tensor(test_data)
# labels = torch.Tensor(labels)

# test = TensorDataset(test_data, labels)

test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=1000,
        num_workers=0,
        shuffle=False
)

In [ ]:
y_pred, y_gt = evaluate(model, test_loader)
y_pred, y_gt

In [ ]:
res = {}
for i in range(10000):
    res[y_gt[i]] = y_pred[i]

In [ ]:
import csv
with open('second_day.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Filename", "Label"])
    for key, item in res.items():
        writer.writerow([key, item])